# Programmers Devmatch 2022

## 1. Fetch Dataset
필요한 라이브러리 설치 및 로드

In [ ]:
# !pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu
!pip3 install pandas
!pip3 install scikit-learn
!pip3 install tqdm

In [1]:
import sys
import torch
import datetime
import shutil
import os.path as osp
sys.path.append('./')
from pytz import timezone
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import *

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from tqdm import tqdm

import pandas as pd
import numpy as np
import os

/root/anaconda3/envs/ecg/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device_type = 'cuda'

if device_type == 'cuda':
    device = 'cuda:4' if torch.cuda.is_available() else 'cpu'
elif device_type == 'mps':
    device = torch.device("mps")


print(device)

cuda:4


## 2. prepare dataset

In [3]:
class trainDatasetBuilder():
    def __init__(self, args, dataset_root = './dataset', dataset1_cv=True, val_ratio=0.1):
        super(trainDatasetBuilder, self).__init__()
        self.args = args

        # encode class
        dataset0_classes = os.listdir(os.path.join(dataset_root, 'dataset0/train/'))
        dataset0_label_encoder = LabelEncoder()
        self.dataset0_classes = dataset0_label_encoder.fit(dataset0_classes)

        dataset1_classes = os.listdir(os.path.join(dataset_root, 'dataset1/train/'))
        dataset1_classes
        dataset1_label_encoder = LabelEncoder()
        self.dataset1_classes = dataset1_label_encoder.fit(dataset1_classes)

        # set path and fetch sample path
        dataset0_dir = os.path.join(dataset_root, 'dataset0/')
        dataset1_dir = os.path.join(dataset_root, 'dataset1/')
        
        self.dataset0_train_dir = dataset0_dir + 'train/'
        self.dataset0_test_dir = dataset0_dir + 'test/'

        self.dataset1_train_dir = dataset1_dir + 'train/'
        self.dataset1_test_dir = dataset1_dir + 'test/'

        # dataset0's training/test datasets & dataloader
        dataset0_X_train, dataset0_y_train = self.create_dataset(mode='train', dataset_dir=self.dataset0_train_dir)
        dataset0_y_train = dataset0_label_encoder.transform(dataset0_y_train)

        dataset0_X_test, dataset0_y_test = self.create_dataset(mode='train', dataset_dir=self.dataset0_test_dir)
        dataset0_y_test = dataset0_label_encoder.transform(dataset0_y_test)

        self.dataset0_train_dataset = TensorDataset(torch.tensor(dataset0_X_train).float(), torch.tensor(torch.from_numpy(dataset0_y_train), dtype=torch.long))
        self.dataset0_test_dataset = TensorDataset(torch.tensor(dataset0_X_test).float(), torch.tensor(torch.from_numpy(dataset0_y_test), dtype=torch.long))

        self.dataset0_train_dataloader = DataLoader(self.dataset0_train_dataset, batch_size=self.args['batch_size'])
        self.dataset0_test_dataloader= DataLoader(self.dataset0_test_dataset, batch_size=self.args['batch_size'])

        # dataset1's training datasets & dataloader
        dataset1_X_train, dataset1_y_train = self.create_dataset(mode='train', dataset_dir=self.dataset1_train_dir)
        dataset1_y_train = dataset1_label_encoder.transform(dataset1_y_train)


        if dataset1_cv == True:
            X_train, X_test, y_train, y_test = train_test_split(dataset1_X_train, dataset1_y_train, test_size=val_ratio, random_state=42)
            self.dataset1_train_dataset = TensorDataset(torch.tensor(X_train).float(), torch.tensor(torch.from_numpy(y_train), dtype=torch.long))
            self.dataset1_test_dataset = TensorDataset(torch.tensor(X_test).float(), torch.tensor(torch.from_numpy(y_test), dtype=torch.long))

            self.dataset1_train_dataloader = DataLoader(self.dataset1_train_dataset, batch_size=self.args['batch_size'])
            self.dataset1_test_dataloader= DataLoader(self.dataset1_test_dataset, batch_size=self.args['batch_size'])
        else:
            self.dataset1_train_dataset = TensorDataset(torch.tensor(dataset1_X_train).float(), torch.tensor(torch.from_numpy(dataset1_y_train), dtype=torch.long))
            self.dataset1_train_dataloader = DataLoader(self.dataset1_train_dataset, batch_size=self.args['batch_size'])
    
    def create_dataset(self, mode='train', dataset_dir=None):
        if mode == 'train':
            X_lst, y_lst = [], []
            labels = os.listdir(dataset_dir)
            labels = [file for file in labels if not file.startswith ('.')] #.DS_Store 제외

            for label in labels:
                file_list = os.listdir(dataset_dir + label + '/')
                file_list = [file for file in file_list if not file.startswith ('.')] #.DS_Store 제외

                for f in file_list:
                    temp = pd.read_csv(dataset_dir + label + '/' + f)
                    X_lst.append(torch.from_numpy(temp.values))
                    y_lst.append(label)
                X_data = pad_sequence(X_lst, batch_first=True)

            return X_data, y_lst
        else:
            raise NotImplementedError('Invalid mode input')


            


## 3. define models
- model 1: transformer
- model 2: resnet-18, 34


### 3.1 define transformer model

In [4]:
# 모델 생성 작성
import math
import copy
import torch
import torch.nn as nn

from torch.nn import TransformerEncoder, TransformerEncoderLayer

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term) 
        pe[:, 1::2] = torch.cos(position * div_term) 
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x


class SelfAttentionPooling(nn.Module):
    """
    Implementation of SelfAttentionPooling 
    Original Paper: Self-Attention Encoding and Pooling for Speaker Recognition
    https://arxiv.org/pdf/2008.01077v1.pdf
    """
    def __init__(self, input_dim):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(input_dim, 1)
        
    def forward(self, batch_rep):
        """
        input:
            batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension
        
        attention_weight:
            att_w : size (N, T, 1)
        
        return:
            utter_rep: size (N, H)
        """
        softmax = nn.functional.softmax
        att_w = softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)

        return utter_rep

class Transformer(nn.Module):
  def __init__(self, d_model=256, in_channels=12, nhead=4, dim_feedforward=1024, nlayers=4, n_class=1, dropout=0.1, dropout_other=0.1):
    super(Transformer, self).__init__()
    encoder_layers = TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
    self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
    self.transformer_encoder2 = TransformerEncoder(encoder_layers, nlayers)
    self.pos_encoder = PositionalEncoding(d_model, dropout)
    self.pos_encoder2 = PositionalEncoding(d_model, dropout)
    self.self_att_pool = SelfAttentionPooling(d_model)
    self.self_att_pool2 = SelfAttentionPooling(d_model)

    self.decoder = nn.Sequential(nn.Linear(d_model, d_model), 
                                       nn.Dropout(dropout_other),
                                       nn.Linear(d_model, d_model), 
                                       nn.Linear(d_model, 64))
    self.decoder2 = nn.Sequential(nn.Linear(d_model, d_model), 
                                       nn.Dropout(dropout_other),
                                      #  nn.Linear(d_model, d_model), 
                                       nn.Linear(d_model, 64))

    self.top_layer1 = nn.Sequential(
                          nn.BatchNorm1d(in_channels),
                          nn.ReLU(inplace=True),
                          nn.Conv1d(in_channels, 32, kernel_size=50, stride=2),
                      )

    self.top_layer2 = nn.Sequential(
                          nn.BatchNorm1d(32),
                          nn.ReLU(inplace=True),
                          nn.Conv1d(32, 64, kernel_size=15, stride=2),
                      )

    self.top_layer3 = nn.Sequential(
                          nn.BatchNorm1d(64),
                          nn.ReLU(inplace=True),
                          nn.Conv1d(64, 128, kernel_size=15, stride=2),
                      )

    self.top_layer4 = nn.Sequential(
                          nn.BatchNorm1d(128),
                          nn.ReLU(inplace=True),
                          nn.Conv1d(128, d_model, kernel_size=15, stride=2),
                      )

    self.bottom_linear = nn.Sequential(
                                 nn.Linear(64, 32),
                                 nn.ReLU(inplace=True),
                                 nn.Dropout(dropout),
                                 nn.Linear(32, n_class), 
                             )

    self.lstm = nn.LSTM(input_size=d_model, hidden_size=d_model//2, num_layers=1,
                                batch_first=False, bidirectional=True)

  def _get_clones(self, module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

  def forward(self, src):   
    src = src.permute(0, 2, 1)
    src = src.squeeze(1) # [batch, 12, 4096]
    src = self.top_layer1(src)
    src = self.top_layer2(src)
    src = self.top_layer3(src)
    src = self.top_layer4(src) # [batch, 256, 241]
    src = src.permute(2, 0, 1) # [241, batch, 256]
    
    # Positinal embedding for ecg signal 
    src = self.pos_encoder(src) # [sequence, batch, embedding]
    
    # Encoding & attention, pool 
    output = self.transformer_encoder(src)
    output = output.permute(1,0,2)
    output = self.self_att_pool(output)
    output = self.decoder(output) # [10, 64]
    
    logits = self.bottom_linear(output)
      
    return logits

### 3.2 define resnet-18, 34 model

In [5]:
class BasicBlock1d(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock1d, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=7, stride=stride, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class ResNet1d(nn.Module):
    def __init__(self, block, layers, input_channels=12, inplanes=64, num_classes=2):
        super(ResNet1d, self).__init__()
        self.inplanes = inplanes
        self.conv1 = nn.Conv1d(input_channels, self.inplanes, kernel_size=15, stride=2, padding=7, bias=False)
        self.bn1 = nn.BatchNorm1d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(BasicBlock1d, 64, layers[0])
        self.layer2 = self._make_layer(BasicBlock1d, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(BasicBlock1d, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(BasicBlock1d, 512, layers[3], stride=2)
        self.adaptiveavgpool = nn.AdaptiveAvgPool1d(1)
        self.adaptivemaxpool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(512 * block.expansion * 2, num_classes)
        self.dropout = nn.Dropout(0.2)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x1 = self.adaptiveavgpool(x)
        x2 = self.adaptivemaxpool(x)
        x = torch.cat((x1, x2), dim=1)
        x = x.view(x.size(0), -1)
        return self.fc(x)


def resnet18(**kwargs):
    model = ResNet1d(BasicBlock1d, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    model = ResNet1d(BasicBlock1d, [3, 4, 6, 3], **kwargs)
    return model

## 4. Train model for transfer learning
`dataset0`을 이용해 `dataset1`을 전이학습하기 위한 best 모델을 훈련합니다. 
### 4.1 define hparams

In [6]:
# define hparams, optimizer, lr with lr_scheduler
args = {'modelname':'devmatch_transformer', 
        'dataset_root' : '/home/mgl/project_src/2022devmatch/dataset',
        'experiment_root':'./experiment',
        'stage':'base', # {base, transfer_learning, both}
        'vis_type' : 'none',
        'batch_size' : 64, 
        'lr' : 0.0001, 
        'epochs' : 500, 
        'dataset0_n_class' : 15, 
        'dataset1_n_class' : 11, 
        'input_channels' : 6, 
        'which_loss' : 'ce',
        'which_optimizer' : 'adamw',
        'scheduler_type' : 'ReduceLROnPlateau',
        'val_interval':1,
        'model_type' : 'transformer' # {'transformer', 'resnet18', 'resnet34'}
}


### 4.2 define trainer 

In [7]:
class Trainer():
    def __init__(self, args, device):
        super(Trainer, self).__init__()
        self.args = args
        self.device = device
        self.step = 0
        self.best_f1 = 0.0
        self.best_auc = 0.0
        self.best_metric_epoch = -1
        self.set_experiment(self.args)

        # Build Dataloader
        self.datasetbuilder = trainDatasetBuilder(args=self.args, dataset_root = self.args['dataset_root'])
        print('Batch size: {}, lr: {}'.format(self.args['batch_size'], self.args['lr']))
        print("Data Loaded")

        # Create Model Instance & Send model to device
        if self.args['model_type'] == 'transformer':
            self.model = Transformer(d_model=128, in_channels=self.args['input_channels'], nhead=4, dim_feedforward=1024, nlayers=4, n_class=self.args['dataset0_n_class'], dropout=0.25, dropout_other=0.25).to(device)
        elif self.args['model_type'] == 'resnet18':
            self.model = resnet18(input_channels=self.args['input_channels'], num_classes=self.args['dataset0_n_class']).to(device)
        elif self.args['model_type'] == 'resnet34':
            self.model = resnet34(input_channels=self.args['input_channels'], num_classes=self.args['dataset0_n_class']).to(device)


        # Define loss function & send to device 
        self.loss_function = self.lossbuilder(which_loss=self.args['which_loss']).to(self.device)
        
        # Define Optimizer & scheduler
        self.optimizer = self.define_optimizer(self.model, which_optimizer=self.args['which_optimizer'])
        self.lr_scheduler = self.define_scheduler(self.optimizer, scheduler_type='ReduceLROnPlateau')
        
        # Track Model Params via Wandb
        if args['vis_type'] == 'wandb':
            wandb.watch(self.model)
        
        # Train model
        if self.args['stage'] == 'base':
            self.train_model(train_dataloader=self.datasetbuilder.dataset0_train_dataloader, val_dataloader=self.datasetbuilder.dataset0_test_dataloader)
        elif self.args['stage'] == 'transfer_learning':
            self.train_model(train_dataloader=self.datasetbuilder.dataset1_train_dataloader, val_dataloader=self.datasetbuilder.dataset1_test_dataloader)
        elif self.args['stage'] == 'both':
            self.train_model(train_dataloader=self.datasetbuilder.dataset0_train_dataloader, val_dataloader=self.datasetbuilder.dataset0_test_dataloader)
            self.train_model(train_dataloader=self.datasetbuilder.dataset1_train_dataloader, val_dataloader=self.datasetbuilder.dataset1_test_dataloader)
        else:
            raise NotImplementedError('Invalid stage input')

    def set_experiment(self, args):
        self.args = args
        name = self.args['modelname']

        now = datetime.datetime.now(timezone('Asia/Seoul'))
        nowDate = now.strftime('%Y-%m-%d')
        nowTime = now.strftime('%H-%M-%S')
        current_time = now.strftime("%Y_%m_%d_%H_%M_%S")

        # create experiment root

        root = osp.join(args['experiment_root'], name+'/{}'.format(current_time))
        if osp.exists(root):
            shutil.rmtree(root, True)


    def train_model(self, train_dataloader, val_dataloader):
        # Epoch Training
        for epoch_idx in tqdm(iterable=range(0, self.args['epochs']), desc='Epoch Progress', leave=False, file=sys.stdout):
            epoch_idx += 1

            # Set model to train mode (allow gradient flows)
            self.model.train()
            epoch_loss = 0

            # Initialize optimizer
            self.optimizer.zero_grad()

            # Batch step
            for b_idx, batch in enumerate(tqdm(iterable=train_dataloader, desc='Batch Progress', leave=False), start=1):
                self.step = b_idx

                ecg_signal, ecg_label = batch
                ecg_signal, ecg_label = ecg_signal.to(self.device), ecg_label.to(self.device)  

                # Create Model Prediction & loss calculation
                output = self.model(src=ecg_signal) # [64, 15]

                loss = self.loss_function(output, ecg_label)
                loss.backward()
                epoch_loss += loss.item()
                self.optimizer.step()

            # Logging Metrics
            epoch_loss /= self.step
            print(f"epoch {epoch_idx} average loss: {epoch_loss:.4f}")
            
            # Validate Model 
            val_loss, avg_f1 = self.validation_model(epoch_idx=epoch_idx, epoch_loss=epoch_loss, dataloader=val_dataloader)

            # Update loss scheduler
            if self.lr_scheduler is not None:
                current_lr = self.optimizer.param_groups[0]['lr']
                print('Current lr: {:.5f}'.format(current_lr))
                self.lr_scheduler.step(val_loss)

            if self.args['vis_type'] == 'wandb':
                wandb.summary['current epoch'] = epoch_idx
                wandb.log({'training loss': epoch_loss})
                wandb.log({'validation loss': val_loss})
                wandb.log({'validation avg_f1': avg_f1})
                wandb.log({'current lr': current_lr})
                

        print("Train completed, Best_metric - F1: {:.3f}, AUC: {:.3f}, at epoch: {}".format(self.best_f1, self.best_auc, self.best_metric_epoch))
        # self.plot_data(self.args)


    def validation_model(self, epoch_idx, epoch_loss, dataloader):
        val_loss = 0
        val_f1_score = 0

        if (epoch_idx) % self.args['val_interval'] == 0:
            self.model.eval()

            with torch.no_grad():
                for v_b_idx, val_data in enumerate(dataloader):
                    self.val_step = v_b_idx
                    ecg_signal, ecg_label = val_data
                    ecg_signal, ecg_label = ecg_signal.to(self.device), ecg_label.to(self.device)
                    
                    # whether use rri feature
                    val_output = self.model(src=ecg_signal)  # data: [64, 15], label: [64]

                    # Compute metric
                    loss = self.loss_function(val_output, ecg_label)
                    val_loss += loss.item()

                    _, preds = torch.max(val_output, 1)
                    print('sibal1', ecg_label.data.cpu().numpy().shape)
                    print('sibal2', preds.data.cpu().numpy().shape)
                    val_f1_score += f1_score(ecg_label.data.cpu().numpy(), preds.data.cpu().numpy())
                    print('sibal score', val_f1_score)
                
                val_loss /= self.val_step
                val_f1_score /= self.val_step
                    
                # Save best model 
                best = val_f1_score > self.best_f1 or val_f1_score > self.best_auc
                if best:
                    self.best_f1 = val_f1_score
                    self.best_metric_epoch = epoch_idx

                    self.save_model(save_dir=self.args['paths']['model_save_dir'], 
                                    model=self.model, 
                                    optimizer=self.optimizer, 
                                    epoch=epoch_idx, 
                                    f1_score=val_f1_score, 
                                    loss=epoch_loss, 
                                    best=best)
                
                # Logging Metrics
                print(f"current epoch: {epoch_idx} current mean F1-score: {val_f1_score:.4f}" 
                      f"\nbest mean F1-score: {self.best_f1:.4f} "
                      f"at epoch: {self.best_metric_epoch}")

                return val_loss, val_f1_score
        

    def define_optimizer(self, model, which_optimizer):
        if which_optimizer == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=self.args['lr'], weight_decay=1e-3)
        elif which_optimizer == 'adamw':
            optimizer = torch.optim.AdamW(model.parameters(), lr=self.args['lr'], betas=([0.9, 0.98]), weight_decay=1e-6)
        else:
            raise NotImplementedError('Invalid optimizer type input.')

        print('Defined optimizer: {}'.format(type(optimizer)))

        return optimizer


    def define_scheduler(self, optimizer, scheduler_type):        
        if scheduler_type == 'StepLR':
            scheduler = StepLR(optimizer, step_size=50, gamma=0.5)
        elif scheduler_type == 'MultiStepLR':
            scheduler = MultiStepLR(optimizer, milestones=[200, 350], gamma=0.5)
        elif scheduler_type == 'ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, cooldown=10)
        elif scheduler_type == 'CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.001)
        elif scheduler_type == 'CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=150, T_mult=1, eta_max=0.1,  T_up=10, gamma=0.5)
        elif scheduler_type == 'None':
            print('No Scheduler Initialized.')
            return None
        else:
            print('Invalid scheduler type input.')
            raise NotImplementedError('LR Scheduler Error')

        print('{} scheduler initialized.'.format(type(scheduler)))

        return scheduler
    

    def lossbuilder(self, which_loss):
        # Define Loss Functions
        if which_loss == 'ce': # multi-class classification
            loss = nn.CrossEntropyLoss()
        elif which_loss == 'bce_logit': # single(one-class) classification 
            loss = nn.BCEWithLogitsLoss()
        else:
            print('Invalid loss input.')
            raise NotImplementedError
        
        print('Defined loss function: {}'.format(type(loss)))
        
        return loss


    def save_model(self, save_dir, model, optimizer, epoch, f1_score: float, loss: float, best=False):
    
        save_dict = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": loss,
        }

        epoch = '{0:04d}'.format(epoch)
        loss = '{:.3f}'.format(loss)
        f1_score = '{:.3f}'.format(f1_score)
        auc = '{:.3f}'.format(auc)

        if best:
            save_path = os.path.join(save_dir, "best_f1{}__epoch{}.pth".format(f1_score, self.best_metric_epoch))
            torch.save(save_dict, save_path)
            print("Best model updated")

        print("Save model to {}".format(save_path))

    def get_lr(self, optimizer):
        for param_group in optimizer.param_groups:
            return param_group['lr']


In [8]:
Trainer(args, device)

/tmp/ipykernel_1758332/700563044.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.dataset0_train_dataset = TensorDataset(torch.tensor(dataset0_X_train).float(), torch.tensor(torch.from_numpy(dataset0_y_train), dtype=torch.long))
/tmp/ipykernel_1758332/700563044.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.dataset0_test_dataset = TensorDataset(torch.tensor(dataset0_X_test).float(), torch.tensor(torch.from_numpy(dataset0_y_test), dtype=torch.long))
/tmp/ipykernel_1758332/700563044.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(so

Batch size: 64, lr: 0.0001
Data Loaded
Defined loss function: <class 'torch.nn.modules.loss.CrossEntropyLoss'>
Defined optimizer: <class 'torch.optim.adamw.AdamW'>
<class 'torch.optim.lr_scheduler.ReduceLROnPlateau'> scheduler initialized.
Epoch Progress:   0%|          | 0/500 [00:00<?, ?it/s]

/tmp/ipykernel_1758332/2906386875.py:51: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_w = softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)


epoch 1 average loss: 2.5522


ValueError: pos_label=1 is not a valid label. It should be one of [0, 13]

## 5. Evaluation model (make submission)

### 5.1 make dataset1 testloader

In [ ]:
"""
해당 셀의 코드 중 dataloader 부분의 `shuffle=False` 는 수정하면 안됩니다.
"""

dataset1_dir = '/home/mgl/project_src/2022devmatch/dataset/dataset1/'
dataset1_test_dir = dataset1_dir + 'test/'

dataset1_test_X = []
test_list = os.listdir(dataset1_test_dir)
for f in test_list:
  temp = pd.read_csv(dataset1_test_dir + f)
  dataset1_test_X.append(torch.from_numpy(temp.values))

dataset1_test_X = pad_sequence(dataset1_test_X, batch_first=True)
dataset1_test_dataset = TensorDataset(torch.Tensor(dataset1_test_X))
dataset1_test_dataloader = DataLoader(dataset1_test_dataset, batch_size=256, shuffle=False)


In [ ]:
# dataset1의 test 데이터를 사용한 일반화 성능 확인

model.eval()
predicted = []
with torch.no_grad():
  for idx, x in enumerate(dataset1_test_dataloader):
    x = x.permute(0, 2, 1).contiguous().to(device)

    optimizer.zero_grad()
    output = model(x.float())

    _, preds = torch.max(output, 1)
    predicted.extend(preds.cpu().numpy())
  torch.cuda.empty_cache()

pd_preds = pd.DataFrame(predicted, columns=['predicted value'])
pd_preds.to_csv('submission.csv')
pd_preds.head()